In [1]:
# imports

import os
import glob
import gradio as gr
import google.generativeai as genai
from dotenv import load_dotenv

In [2]:
# imports for langchain and Chroma and plotly

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document

from langchain.vectorstores import FAISS

from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [3]:
# price is a factor for our company, so we're going to use a low cost model

MODEL = "gemini-2.0-flash"
db_name = "vector_db"
path = "stock_analysis/reports_v2/*"

In [4]:
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY', 'your-key-if-not-using-env')
genai.configure(api_key=GOOGLE_API_KEY)

In [5]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase

folders = glob.glob(path)
print("folders ", folders)
# With thanks to CG and Jon R, students on the course, for this fix needed for some users
text_loader_kwargs = {'encoding': 'utf-8'}
# If that doesn't work, some Windows users might need to uncomment the next line instead
# text_loader_kwargs={'autodetect_encoding': True}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

folders  ['stock_analysis/reports_v2\\microcap_250', 'stock_analysis/reports_v2\\midcap_150', 'stock_analysis/reports_v2\\nifty_50', 'stock_analysis/reports_v2\\smallcap_250']


In [6]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [7]:
len(chunks)

2116

In [8]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: midcap_150, microcap_250, smallcap_250, nifty_50


In [9]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)

In [10]:
# Create our FAISS vectorstore!

vectorstore = FAISS.from_documents(documents=chunks, embedding=embeddings)

total_vectors = vectorstore.index.ntotal
dimensions = vectorstore.index.d

print(f"There are {total_vectors} vectors with {dimensions:,} dimensions in the vector store")

There are 2116 vectors with 768 dimensions in the vector store


In [11]:
# Prework for FAISS
vectors = []
documents = []
doc_types = []
colors = []
color_map = {'nifty_50':'blue', 'midcap_150':'green', 'smallcap_250':'red', 'microcap_250':'orange'}

for i in range(total_vectors):
    vectors.append(vectorstore.index.reconstruct(i))
    doc_id = vectorstore.index_to_docstore_id[i]
    document = vectorstore.docstore.search(doc_id)
    documents.append(document.page_content)
    doc_type = document.metadata['doc_type']
    doc_types.append(doc_type)
    colors.append(color_map[doc_type])

vectors = np.array(vectors)

In [12]:
# We humans find it easier to visalize things in 2D!
# Reduce the dimensionality of the vectors to 2D using t-SNE
# (t-distributed stochastic neighbor embedding)

tsne = TSNE(n_components=2, random_state=42, perplexity=min(25, len(vectors) - 1))
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [13]:
# Let's try 3D!

tsne = TSNE(n_components=3, random_state=42, perplexity=min(25, len(vectors) - 1))
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [14]:
###RAG Pipeline

In [15]:
print(f"Total chunks in vectorstore: {len(chunks)}")
print(f"Total unique stocks: 621")
print(f"Average chunks per stock: {len(chunks)/621:.2f}")

# Calculate recommended k value
recommended_k = len(chunks)  # Start with maximum possible chunks
print(f"Recommended k value: {recommended_k}")

Total chunks in vectorstore: 2116
Total unique stocks: 621
Average chunks per stock: 3.41
Recommended k value: 2116


In [16]:
# create a new Chat with OpenAI
llm = ChatGoogleGenerativeAI(model=MODEL, temperature=0.7, google_api_key = GOOGLE_API_KEY)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG; k is how many chunks to use
retriever = vectorstore.as_retriever(search_kwargs={"k": int(recommended_k/2)})

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

C:\Users\anand\AppData\Local\Temp\ipykernel_21880\1276407914.py:5: LangChainDeprecationWarning:

Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/



In [17]:
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [18]:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
